In [2]:
# -*- coding: utf-8 -*-
#!/usr/bin/python

import RPi.GPIO as GPIO
import os
import smbus
import time
import pickle

from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
output_notebook()

class ADXL345():
    DevAdr = 0x53
    myBus = ""
    if GPIO.RPI_INFO['P1_REVISION'] == 1:
        myBus = 0
    else:
        myBus = 1
    b = smbus.SMBus(myBus)
        
    def setUp(self):
        self.b.write_byte_data(self.DevAdr, 0x2C, 0x0B) # BandwidthRate　originally 0B 
        self.b.write_byte_data(self.DevAdr, 0x31, 0x00) # DATA_FORMAT 10bit 2g
        self.b.write_byte_data(self.DevAdr, 0x38, 0x00) # FIFO_CTL OFF
        self.b.write_byte_data(self.DevAdr, 0x2D, 0x08) # POWER_CTL Enable

    def getData(self):
        output = []
        data = self.b.read_i2c_block_data(self.DevAdr, 0x32, 6)
        for i in range(0,6,2):
            sign = data[i + 1] & 0x80
            tmp = data[i + 1] & 0x7F
            tmp = tmp << 8
            tmp = tmp | data[i]

            if sign > 0:
                output.append(tmp - 32768)
            else:
                output.append(tmp)

        return output

def main():
    myADXL345 = ADXL345()
    myADXL345.setUp()

    SAMPLING_CYCLE = 0.01 # in seconds
    MONITOR_TIME = 10 # in seconds
    NO_OF_SAMPLES = int(MONITOR_TIME * ( 1.0  / SAMPLING_CYCLE ))

    print NO_OF_SAMPLES

    print myADXL345.getData()
    #print myADXL345.getValueX()

    X=[]
    Y=[]
    Z=[]
    print "Start"
    start = time.time()
    for a in range(NO_OF_SAMPLES):
        x, y, z = myADXL345.getData()
        X.append(x)
        Y.append(y)
        Z.append(z)
        time.sleep(SAMPLING_CYCLE)

    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
    print elapsed_time / NO_OF_SAMPLES
    print "Stop"

    t=[float(n) for n in range(0,NO_OF_SAMPLES)]

    with open('X.pkl', 'wb') as f:
        pickle.dump(X, f)
    with open('Y.pkl', 'wb') as f:
        pickle.dump(Y, f)
    with open('Z.pkl', 'wb') as f:
        pickle.dump(Z, f)
    
if __name__ == "__main__":
    # execute only if run as a script
    main()
    
    # plot figure 
    pp = figure(tools='xwheel_zoom,xpan',
    title="",
    x_axis_label='t[s]',
    y_axis_label='Vibration')
    pp.line(t, X,legend="Vibration", line_width=1, line_color = "blue")
    pp.line(t, Y,legend="Vibration", line_width=1, line_color = "red")
    pp.line(t, Z,legend="Vibration", line_width=1, line_color = "green")
    output_file("robot.html")
    show(pp)

Loading BokehJS ...

1000
[0, 55, 271]
Start
elapsed_time:11.5277459621[sec]
0.0115277459621
Stop


# オフセット測定

In [6]:
import numpy as np

with open('X.pkl', 'rb') as f:
    X = pickle.load(f)

with open('Y.pkl', 'rb') as f:
    Y = pickle.load(f)
    
with open('Z.pkl', 'rb') as f:
    Z = pickle.load(f)

X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)
print Z[:5]
print X.mean()

[256 271 270 271 270]
-2.815


# 加速度を、移動平均

In [18]:
import numpy as np

with open('X.pkl', 'rb') as f:
    X = pickle.load(f)

with open('Y.pkl', 'rb') as f:
    Y = pickle.load(f)
    
with open('Z.pkl', 'rb') as f:
    Z = pickle.load(f)

X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)

X = X - 11.2283
Y = Y - Y.mean()
Z = Z - Z.mean()

X_v = np.zeros(NO_OF_SAMPLES)
Y_v = np.zeros(NO_OF_SAMPLES)
Z_v = np.zeros(NO_OF_SAMPLES)

for i in range(0, NO_OF_SAMPLES -1):
    X_v[i+1] = 0.90 * X_v[i] + 0.1 * X[i]
    
for i in range(0, NO_OF_SAMPLES -1):
    Y_v[i+1] = 0.90 * Y_v[i] + 0.1 * Y[i]

for i in range(0, NO_OF_SAMPLES -1):
    Z_v[i+1] = 0.90 * Z_v[i] + 0.1 * Z[i]

pp = figure(tools='xwheel_zoom,xpan',
title="",
x_axis_label='t[s]',
y_axis_label='Vibration')
pp.line(t, X_v,legend="Vibration", line_width=2, line_color = "blue")
#pp.line(t, X,legend="Vibration", line_width=1, line_color = "red")
pp.line(t, Y_v,legend="Vibration", line_width=1, line_color = "green")
pp.line(t, Z_v,legend="Vibration", line_width=1, line_color = "yellow")
output_file("robot.html")
show(pp)